<a href="https://colab.research.google.com/github/barrosm/text_mining/blob/master/Aula_20191123_similaridade_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercícios de medidas de similaridade de texto #

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
workdir_path = '/content/drive/My Drive/aula_20191123_leonardo/' #MODIFICAR!!! #Caminho para o seu workspace
os.chdir(workdir_path)

## Introdução ##

Usaremos [um conjunto de dados de letras de músicas do Kaggle] (https://www.kaggle.com/mousehead/songlyrics) para identificar músicas com letras semelhantes. O conjunto de dados contém artistas, músicas e letras para mais de 55 mil músicas, mas hoje vamos nos concentrar nas músicas de um grupo em particular - os Beatles.

O código a seguir ajudará você a carregar os dados e a se preparar para este exercício.

In [7]:
!pip install nltk

In [0]:
import nltk
import pandas as pd

In [9]:
data = pd.read_csv('songdata.csv')
data.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


## Desafio  1 ##

* Filtre o conjunto de dados da letra para selecionar apenas músicas dos Beatles.
* Quantas músicas existem no total pelos Beatles?
* Dê uma olhada nas letras da primeira música.

In [10]:
# Veja apenas as músicas dos Beatles
beatles = data[data.artist=='The Beatles']
beatles.head()

,artist,song,link,text
1198,The Beatles,A Shot Of Rhythm And Blues,/b/beatles/a+shot+of+rhythm+blues_20014867.html,"Well, if your hands start a-clappin' \r\nAnd ..."
1199,The Beatles,Across The Universe,/b/beatles/across+the+universe_10026507.html,Words are flowing out like \r\nEndless rain i...
1200,The Beatles,All I've Got To Do,/b/beatles/all+ive+got+to+do_10026646.html,"Whenever I want you around, yeah \r\nAll I go..."
1201,The Beatles,And I Love Her,/b/beatles/and+i+love+her_10026463.html,I give her all my love \r\nThat's all I do \...
1202,The Beatles,And Your Bird Can Sing,/b/beatles/and+your+bird+can+sing_10026364.html,You tell me that you've got everything you wan...


In [11]:
# Verifique o número de músicas dos Beatles no conjunto de dados
data.artist.value_counts().loc[['The Beatles']]

The Beatles    178
Name: artist, dtype: int64

In [12]:
# Veja a letra da primeira música
beatles.iloc[0,3]

"Well, if your hands start a-clappin'  \r\nAnd your fingers start a-poppin'  \r\nAnd your feet start a-movin' around  \r\nAnd if you start to swing and sway  \r\n  \r\nWhen the band starts to play  \r\nA real cool way out sound  \r\nAnd if you get to can't help it and you can't sit down  \r\nYou feel like you gotta move around  \r\n  \r\nYou get a shot of rhythm and blues.  \r\nWith just a little rock and roll on the side  \r\nJust for good measure.  \r\nGet a pair of dancin' shoes  \r\n  \r\nWell, with your lover by your side  \r\nDon't you know you're gonna have a rockin' time, see'mon!  \r\nDon't you worry 'bout a thing  \r\nIf you start to dance and sing  \r\n  \r\nAnd chills come up on you  \r\nAnd if the rhythm finally gets you and the beat gets you too  \r\nWell, here's something for you to do  \r\n  \r\nGet a shot of rhythm and blues  \r\nWith just a little rock and roll on the side  \r\nJust for good measure  \r\nGet a pair of dancin' shoes  \r\n  \r\nWell, with your lover by 

## Desafio 2 ##


Aplique as seguintes etapas de pré-processamento:
* Observe os caracteres '\ n' (nova linha) nas letras. Remova-os usando expressões regulares.
* Remova todas as palavras com números usando expressões regulares.
* Crie uma matriz de termos de documento usando o Count Vectorizer, com cada linha como uma música e cada coluna como uma palavra nas letras. Faça com que o Count Vectorizer remova todas as palavras de parada também.

Nota: O Count Vectorizer remove automaticamente a pontuação e coloca todos os caracteres em minúsculas.

In [13]:
# Remover caracteres da letra
import re

newline = lambda x: re.sub('\n', ' ', x) # remove \n
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x) # remove alphanumeric words

corpus = beatles.text.map(newline).map(alphanumeric)
corpus.head()

1198    Well, if your hands start a-clappin'  \r And y...
1199    Words are flowing out like  \r Endless rain in...
1200    Whenever I want you around, yeah  \r All I got...
1201    I give her all my love  \r That's all I do  \r...
1202    You tell me that you've got everything you wan...
Name: text, dtype: object

In [0]:
# Crie uma matriz de termos de documento usando o Count Vectorizer com as palavras de parada ativadas para inglês
from sklearn.feature_extraction.text import CountVectorizer
          
cv = CountVectorizer(stop_words="english")
X = cv.fit_transform(corpus).toarray()

dt = pd.DataFrame(X, columns=cv.get_feature_names()).set_index(beatles.song)
dt.head()

## Desafio  3 ##

* Dê uma olhada na letra da música "Imagine".
* Qual música é a mais parecida com a música "Imagine"?
      * Use a semelhança de cosseno para calcular a semelhança
      * Use o Count Vectorizer para codificar numericamente a letra
* Encontre a música mais parecida usando o TF-IDF Vectorizer.

Compare a música mais semelhante das saídas do Count Vectorizer e do TF-IDF Vectorizer.

In [0]:
# Defina as opções de exibição para que as letras não sejam cortadas
pd.set_option('display.max_colwidth', -1)

In [0]:
#  letra Imagine
beatles[beatles.song=='Imagine'].text

In [0]:
# imagine "letra no formato Count Vectorizer"
imagine = list(dt.loc['Imagine'])
imagine[:20]

In [0]:
# Definir o cálculo de similaridade de cosseno
from numpy import dot
from numpy.linalg import norm

cosine = lambda v1, v2: dot(v1, v2) / (norm(v1) * norm(v2))

In [0]:
# Calcular todas as semelhanças e classificar pelas mais semelhantes
similarity = [cosine(imagine, song) for song in X]
sorted(list(zip(similarity, beatles.song)), reverse=True)[1:6]

In [0]:
# Crie o vetorizador TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
          
cv_tfidf = TfidfVectorizer(stop_words="english")
X_tfidf = cv_tfidf.fit_transform(corpus).toarray()

dt_tfidf = pd.DataFrame(X_tfidf, columns=cv_tfidf.get_feature_names()).set_index(beatles.song)
dt_tfidf.head()

In [0]:
# Calcule todas as semelhanças e classifique as mais semelhantes usando os dados do TF-IDF
similarity_tfidf = [cosine(imagine, song) for song in X_tfidf]
sorted(list(zip(similarity_tfidf, beatles.song)), reverse=True)[1:6]

As respostas para o Count Vectorizer e TF-IDF Vectorizer estão bem próximas. Nos dois casos, "Eu vou chorar" é a música mais parecida com "Imagine". Quando você olha as letras, pode ver que as duas estão do lado triste das músicas dos Beatles.

In [0]:
# Imagine lyrics
beatles[beatles.song=='Imagine'].text

In [0]:
# I'll Cry Instead lyrics
beatles[beatles.song=='I\'ll Cry Instead'].text

## Desafio 4 ##

Quais são as duas músicas dos Beatles mais parecidas?
    * Usando o Count Vectorizer
    * Usando o TFizer Vectorizer
     
Compare os resultados. Qual vetorizador parece fazer um trabalho melhor?

In [0]:
# Calcular a semelhança de cosseno entre todas as combinações de documentos

In [0]:
# Listar todas as combinações de músicas
from itertools import combinations

pairs = list(combinations(beatles.song.index, 2)) # all song index combos
pairs_0 = list(combinations(range(len(beatles)), 2)) # all index combos starting with (0,1)
song_pairs = [(beatles.song[a_index], beatles.song[b_index]) for (a_index, b_index) in pairs]
song_pairs[:5]

In [0]:
# Calcular a semelhança de cosseno da matriz termo-documento do Count Vectorizer
results = [cosine(X[a_index], X[b_index]) for (a_index, b_index) in pairs_0]
sorted(zip(results, song_pairs), reverse=True)[:5]

In [0]:
# Calcular a semelhança de cosseno da matriz termo-documento do TF-IDF Vectorizer
results = [cosine(X_tfidf[a_index], X_tfidf[b_index]) for (a_index, b_index) in pairs_0]
sorted(zip(results, song_pairs), reverse=True)[:5]


Com o Count Vectorizer, todas as músicas principais contêm o termo "love", com "All You Need Is Love" e "Love Me Do" sendo as mais semelhantes. Isso ocorre porque os Beatles tendem a usar muito o termo "amor" em suas músicas.

Com o TF-IDF Vectorizer, as principais músicas semelhantes são um pouco mais interessantes. Parece que ele realmente pegou uma música duplicada no conjunto de dados das letras com "If I Needed Someone" e "If I Needed Someone To Love". Além disso, "Tenha uma banana!" e "Crinsk Dee Night" são conversas com o apresentador Brian Matthew.

In [0]:
# All You Need Is Love lyrics
beatles[beatles.song=='All You Need Is Love'].text

In [0]:
# Love Me Do lyrics
beatles[beatles.song=='Love Me Do'].text

In [0]:
# If I Needed Someone lyrics
beatles[beatles.song=='If I Needed Someone'].text

In [0]:
# If I Needed Someone To Love lyrics
beatles[beatles.song=='If I Needed Someone To Love'].text

In [0]:
# Have a Banana! lyrics
beatles[beatles.song=='Have A Banana!'].text

In [0]:
# Crinsk Dee Night lyrics
beatles[beatles.song=='Crinsk Dee Night'].text